In [1]:
import pandas as pd
import glob
import os
import pymysql

# path details of all csv files
folder_path = r'C:\Users\HP\Desktop\kani\project_redbus\Redbus_recheck'

# Use glob to find all csv files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

dfs=[]
# Read each csv file and print or store the content
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)   

merged_df.to_csv("all_state_bus_detail.csv", index=False)

### data cleaning and data type changes:
df = pd.read_csv("all_state_bus_detail.csv")


df['Price'] = df['Price'].str.replace('INR', "", regex= False)
df['Price'] = pd.to_numeric(df['Price'], errors = 'coerce')


df['Seats Available'] = df['Seats Available'].str.extract(r'(\d+)')
df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors = 'coerce')

df['Seats Available'] = df['Seats Available'].astype('Int64')
df['Route Name'] = df['Route Name'].astype(str)
df['Route Link'] = df['Route Link'].astype(str)
df['Bus Name'] = df['Bus Name'].astype(str)
df['Bus Type'] = df['Bus Type'].astype(str)
df['Departure Time'] = pd.to_datetime(df['Departure Time'], format='%H:%M').dt.time  # Convert to time
df['Reaching Time'] = pd.to_datetime(df['Reaching Time'], format='%H:%M').dt.time  # Convert to time
df['State'] = df['State'].astype(str)
df= df.dropna()

# Connect to MySQL
try:
    myconnection = pymysql.connect(
        host='127.0.0.1',
        user='root',
        passwd='Nikil@8405',
        database='Red_Bus'
    )
    cursor = myconnection.cursor()
    print("Connection successful!")
except pymysql.OperationalError as e:
    print(f"OperationalError: {e}")
# Creating database:
# Create database red_bus


# SQL query to create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS Bus_routes (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    State TEXT,
    Route_Name TEXT,
    Route_Link TEXT,
    Bus_Name TEXT,
    Bus_Type TEXT,
    Departing_Time TIME,
    Duration TEXT,
    Reaching_Time TIME,
    Star_Rating FLOAT,
    Price DECIMAL(10, 2),
    Seats_Available INT
);
"""

cursor.execute(create_table_query)

print("Table Bus_routes created successfully!")


# Insert data into table
insert_query = """
INSERT INTO Bus_routes (State,Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seats_Available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

data_tuples = [
    (
        row['State'],
        row['Route Name'],
        row['Route Link'],
        row['Bus Name'],
        row['Bus Type'],
        row['Departure Time'],
        row['Duration'],
        row['Reaching Time'],
        row['Rating'],
        row['Price'],
        row['Seats Available']
    )
    for index, row in df.iterrows()
]

# Use executemany for batch inserts
cursor.executemany(insert_query, data_tuples)
myconnection.commit()
print("Data inserted successfully!")

cursor.close()
myconnection.close()

Connection successful!
Table Bus_routes created successfully!
Data inserted successfully!


In [3]:
df = pd.read_csv("all_state_bus_detail.csv")

In [5]:
df.dtypes

State               object
Route Name          object
Route Link          object
Bus Name            object
Bus Type            object
Departure Time      object
Duration            object
Reaching Time       object
Rating             float64
Price               object
Seats Available     object
dtype: object

In [7]:
df['Price'] = df['Price'].str.replace('INR', "", regex= False)
df['Price'] = pd.to_numeric(df['Price'], errors = 'coerce')


df['Seats Available'] = df['Seats Available'].str.extract(r'(\d+)')
df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors = 'coerce')

df['Seats Available'] = df['Seats Available'].astype('Int64')  # Nullable integer
df['Route Name'] = df['Route Name'].astype(str)
df['Route Link'] = df['Route Link'].astype(str)
df['Bus Name'] = df['Bus Name'].astype(str)
df['Bus Type'] = df['Bus Type'].astype(str)
df['Departure Time'] = pd.to_datetime(df['Departure Time'], format='%H:%M').dt.time  # Convert to time
df['Reaching Time'] = pd.to_datetime(df['Reaching Time'], format='%H:%M').dt.time  # Convert to time
df['State'] = df['State'].astype(str)

In [9]:
df.dtypes

State               object
Route Name          object
Route Link          object
Bus Name            object
Bus Type            object
Departure Time      object
Duration            object
Reaching Time       object
Rating             float64
Price              float64
Seats Available      Int64
dtype: object